In [4]:
from pathlib import Path
from train_cnn import EEGDataModule
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
from util import load_split_ids

In [5]:
path_to_split = Path("/home/bc299/icare/artifacts")
train_ids, val_ids, test_ids = load_split_ids(path_to_split)

In [8]:
import matplotlib.pyplot as plt

DEST_DIR = Path("/media/hdd1/i-care/10s")

# Initialize the EEGDataModule
data_module = EEGDataModule(train_ids, val_ids, DEST_DIR)
data_module.setup()

# 1. Check the Length of Datasets
print(f"Number of training segments: {len(data_module.train_dataset)}")
print(f"Number of validation segments: {len(data_module.val_dataset)}")

# 2. Visualize Some Data Points
def visualize_segments(segments, num=5):
    for i in range(min(num, len(segments))):
        plt.figure(figsize=(10, 4))
        plt.plot(segments[i].numpy())
        plt.title(f"Segment {i+1}")
        plt.xlabel("Time Points")
        plt.ylabel("EEG Value")
        plt.show()

# Load some training data
train_segments = next(iter(data_module.train_dataloader()))
print("Visualizing Training Data:")
visualize_segments(train_segments)

# Load some validation data
val_segments = next(iter(data_module.val_dataloader()))
print("Visualizing Validation Data:")
visualize_segments(val_segments)

# 3. Check Batch Sizes
print(f"Training batch size: {train_segments.size(0)}")
print(f"Validation batch size: {val_segments.size(0)}")


FileNotFoundError: [Errno 2] No such file or directory: '/media/hdd1/i-care/10s/0858'